In [2]:
import pandas as pd

In [3]:
df=pd.read_csv('Dog-Cat-QA.csv')
df.head()

,id,Question,Answer
0,0,What are some examples of breed-typical person...,Breed-typical personalities in dogs are develo...
1,1,How do working and herding dogs typically behave?,Working and herding dogs have business-like di...
2,2,Which breed of dogs is known for strong loyalty?,Collies and Akitas are known for their strong ...
3,3,What instincts do guarding dogs typically disp...,Guarding dogs tend to be protective of their t...
4,4,How does breed specificity impact a dog's abil...,Breed specificity affects how well dogs adapt ...


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 583 entries, 0 to 582
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        583 non-null    int64 
 1   Question  583 non-null    object
 2   Answer    583 non-null    object
dtypes: int64(1), object(2)
memory usage: 13.8+ KB


In [5]:
df.columns

Index(['id', 'Question', 'Answer'], dtype='object')

In [9]:
# !wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py


--2024-09-09 12:21:52--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: 'minsearch.py'

     0K ...                                                   100% 12.1M=0s

2024-09-09 12:21:52 (12.1 MB/s) - 'minsearch.py' saved [3832/3832]



In [6]:
import minsearch


In [29]:
# documents = df[['Question', 'Answer']].to_dict(orient='records')

documents = df.to_dict(orient='records')


In [8]:
documents[0]

{'Question': 'What are some examples of breed-typical personalities in dogs?',
 'Answer': 'Breed-typical personalities in dogs are developed through generations of selection for certain traits. For example, sporting dogs are adventurous and responsive to humans, while hounds tend to be more independent and aloof.'}

In [30]:
index = minsearch.Index(
     text_fields=["Question", "Answer"],
     keyword_fields=['id']
)

In [31]:
q='which cat breed is the most friendly?'

In [32]:
index.fit(documents)


## RAG flow

In [33]:
from dotenv import load_dotenv
load_dotenv()
from openai import OpenAI
client = OpenAI()


In [12]:
response = client.chat.completions.create(
    model='gpt-4o-mini',
    messages=[{"role": "user", "content": q}]
)

print(response.choices[0].message.content)

While individual cat personalities can vary widely, certain breeds are often noted for their friendly and sociable nature. Some of the most commonly recognized friendly cat breeds include:

1. **Ragdoll**: Known for their gentle and laid-back disposition, Ragdolls tend to enjoy human companionship and often follow their owners around.

2. **Maine Coon**: This breed is known for its friendly and affectionate nature. They are often described as "gentle giants" and are great with families.

3. **Siamese**: Siamese cats are social and vocal, often forming strong bonds with their human companions. They thrive on interaction and companionship.

4. **British Shorthair**: Known for their easygoing personality, British Shorthairs are friendly and adaptable, making them great pets for families.

5. **Burmese**: These cats are known for their affectionate and playful nature. They love being around people and enjoy interactive play.

6. **Scottish Fold**: With their distinctive folded ears and swe

In [34]:
def search(query):
    boost = {}

    results = index.search(
        query=query,
        filter_dict={ },
        boost_dict=boost,
        num_results=6
    )

    return results

In [35]:
query = 'which dog breed is the most friendly?'


In [37]:
search_results=search(query)
print(search_results, len(search_results))

[{'id': 2, 'Question': 'Which breed of dogs is known for strong loyalty?', 'Answer': 'Collies and Akitas are known for their strong sense of loyalty.'}, {'id': 517, 'Question': 'How can I choose the right dog or breed for my lifestyle?', 'Answer': "Consider factors such as the dog's size, energy level, temperament, and grooming needs. Visiting potential dog candidates in person is also recommended."}, {'id': 572, 'Question': ' What is the most important thing to do when your dog is scared of fireworks?', 'Answer': " Staying calm and relaxed is crucial because dogs look to their owners for reassurance. Making a big fuss can reinforce the dog's panic."}, {'id': 164, 'Question': 'What is the most advanced obstacle in feline agility training?', 'Answer': 'Weaving around poles is one of the most advanced obstacles in feline agility training.'}, {'id': 377, 'Question': " Have you thought about the cat's breed and how it aligns with your lifestyle and preferences?", 'Answer': ' Consider which

In [38]:
from openai import OpenAI
client = OpenAI()


In [41]:
def build_pet_care_prompt(query, search_results):
    prompt_template = """
You are a knowledgeable pet care expert specializing in dogs and cats. 
Answer the USER QUESTION based on the INFORMATION from our pet care database. 
Use only the facts from the INFORMATION when answering the USER QUESTION. 
If the information doesn't provide a complete answer, 
say so and suggest seeking professional veterinary advice.

USER QUESTION: {question}

INFORMATION:
{context}

Please provide a clear and concise answer. If the question is about a specific breed or species, make sure to highlight that in your response. If there are any safety concerns or potential health risks mentioned in the context, emphasize those in your answer.
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"Question: {doc['Question']}\nAnswer: {doc['Answer']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [42]:
def llm(prompt,model='gpt-4o-mini'):
    response = client.chat.completions.create(
        model='gpt-4o-mini',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [18]:
# len(documents)

583

In [43]:

def rag(query,model='gpt-4o-mini'):
    search_results = search(query)
    prompt = build_pet_care_prompt(query, search_results)
    answer = llm(prompt,model=model)
    return answer

In [44]:
query = 'What traits define the personality of sporting dog breeds'
answer=rag(query)
print(answer)


The provided information does not specify the personality traits that define sporting dog breeds. To get accurate and comprehensive information about the personality traits of sporting dog breeds, I recommend seeking professional veterinary advice or referring to breed-specific resources.


In [56]:
#rename first column to 'id'


In [21]:
df.head()

,id,Question,Answer
0,0,What are some examples of breed-typical person...,Breed-typical personalities in dogs are develo...
1,1,How do working and herding dogs typically behave?,Working and herding dogs have business-like di...
2,2,Which breed of dogs is known for strong loyalty?,Collies and Akitas are known for their strong ...
3,3,What instincts do guarding dogs typically disp...,Guarding dogs tend to be protective of their t...
4,4,How does breed specificity impact a dog's abil...,Breed specificity affects how well dogs adapt ...


In [58]:
# update df back to file


In [28]:
df=pd.read_csv('Dog-Cat-QA.csv')

In [22]:
df.head(),len(df)

(   id                                           Question  \
 0   0  What are some examples of breed-typical person...   
 1   1  How do working and herding dogs typically behave?   
 2   2   Which breed of dogs is known for strong loyalty?   
 3   3  What instincts do guarding dogs typically disp...   
 4   4  How does breed specificity impact a dog's abil...   
 
                                               Answer  
 0  Breed-typical personalities in dogs are develo...  
 1  Working and herding dogs have business-like di...  
 2  Collies and Akitas are known for their strong ...  
 3  Guarding dogs tend to be protective of their t...  
 4  Breed specificity affects how well dogs adapt ...  ,
 583)

In [45]:
query="What are the signs of stress in cats and dogs?"
answer=rag(query)
print(answer)


Signs of stress in dogs may include excessive yawning, lip-licking, trembling, avoiding or hiding, and a hardening of the eyes. 

Unfortunately, the provided information does not specifically list signs of stress in cats. It's important to consult with a veterinarian for more comprehensive insights on stress signs in cats, along with any potential health implications.


## Retrieval evaluation 

In [46]:
df_questions=pd.read_csv('../data/ground-truth-retrieval.csv')  
df_questions.head()

,id,question
0,0,What are the common personality traits associa...
1,0,Can you explain the typical temperament of hou...
2,0,How do breed characteristics influence a dog's...
3,1,What behavior can I expect from working and he...
4,1,How do herding dogs usually assess their envir...


In [47]:
ground_truth = df_questions.to_dict(orient='records')


In [48]:
ground_truth[0]


{'id': 0,
 'question': 'What are the common personality traits associated with sporting dog breeds?'}

In [50]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)

In [51]:
def minsearch_search(query):
    boost = {}

    results = index.search(
        query=query,
        filter_dict={},
        boost_dict=boost,
        num_results=6
    )

    return results

In [52]:
def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['id']
        results = search_function(q)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }

In [53]:
from tqdm.auto import tqdm

c:\Users\wufen\.virtualenvs\Pet-Care-RAG-Application-DW-wRTOa\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [54]:
evaluate(ground_truth, lambda q: minsearch_search(q['question'] ))


100%|██████████| 1749/1749 [00:09<00:00, 188.34it/s]


{'hit_rate': 0.8547741566609491, 'mrr': 0.6890794739851345}